In [1]:
from datasets import load_dataset,concatenate_datasets, Dataset,DatasetDict
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch

from DMLP.models.my_transformers import MODEL_CLASS
from DMLP.models.models import VAE, DDPM, MLPSkipNet, TransformerNet,VAE_DDPM
from DMLP.train.reconstruction import *
from DMLP.utils.ddpm_schedule import ddpm_schedule
from DMLP.utils.random_init import weights_init_random
from DMLP.train.train_function import train_vae_ddpm


/home/AD/yul080/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/AD/yul080/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/AD/yul080/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
class MyCollator(object):
    def __init__(self, encoder_token, decoder_token):
        self.encoder_token = encoder_token
        self.decoder_token = decoder_token
    def __call__(self, batch):
        input_ids_bert = pad_sequence([torch.tensor(f['bert_token'], dtype=torch.long) for f in batch],
                                  batch_first=True, padding_value=self.encoder_token)
        input_ids_gpt = pad_sequence([torch.tensor(f['gpt2_token'], dtype=torch.long) for f in batch],
                                    batch_first=True, padding_value=self.decoder_token)
        try:
            token_lengths = torch.tensor([[len(f['bert_token']), len(f['gpt2_token'])] for f in batch],
                                        dtype=torch.long)
        except:
            token_lengths = torch.zeros((len(batch), 1091))
            for i in range(len(batch)):
                token_lengths[i, len(batch[i]['gpt2_token'])] = 1
        return (input_ids_bert, input_ids_gpt, token_lengths)

In [16]:
batch_size = 2
tokenizer_encoder = AutoTokenizer.from_pretrained("prajjwal1/bert-small")
tokenizer_decoder = AutoTokenizer.from_pretrained("gpt2-xl")
special_tokens_dict = {'pad_token': '<PAD>', 'bos_token': '<BOS>', 'eos_token': '<EOS>', }
num_added_toks = tokenizer_decoder.add_special_tokens(special_tokens_dict)

bert_pad_token = tokenizer_encoder.pad_token_id
gpt2_pad_token = tokenizer_decoder.pad_token_id
my_collator = MyCollator(bert_pad_token, gpt2_pad_token)
#download data
print("download data")

train_eval_dataset =load_dataset("guangyil/yelp_short_v2")
eval_dataloader =  DataLoader(train_eval_dataset['test'], num_workers=0, collate_fn=my_collator,batch_size=batch_size)
train_dataloader = DataLoader(train_eval_dataset['train'], num_workers=0, collate_fn=my_collator, batch_size=batch_size)

download data


In [15]:
Dataset.from_dict(train_eval_dataset['test'][0])

AttributeError: 'Dataset' object has no attribute 'items'

In [18]:
eval_dataloader

In [6]:
#load mode
latent_size = 128
encoder_model_class = MODEL_CLASS['BertForLatentConnectorAVG']
decoder_model_class = MODEL_CLASS['GPT2ForLatentConnectorNew']

output_dir = "home/AD/yul080/out_temp"

In [9]:
model_decoder = decoder_model_class.from_pretrained("gpt2-xl", latent_size=latent_size,
                                                            latent_as_gpt_emb=True,
                                                            latent_as_gpt_memory=True,local_files_only=False)
model_decoder.resize_token_embeddings(len(tokenizer_decoder))
decoder_n_layer = model_decoder.transformer.config.n_layer
model_decoder.transformer.change_order()

model_encoder = encoder_model_class.from_pretrained("prajjwal1/bert-small", latent_size=latent_size,
                                                        pad_id=tokenizer_encoder.pad_token_id,local_files_only=False)
model_vae = VAE(model_encoder, model_decoder, tokenizer_encoder, tokenizer_decoder, latent_size, output_dir)


# checkpoint = torch.load('/home/AD/yul080/runs/checkpoint-full-2/training.bin',map_location=torch.device('cpu'))
model_vae.apply(weights_init_random)
# model_vae.load_state_dict(checkpoint['model_state_dict'], strict=False) 
model_vae.to('cuda')   

Some weights of GPT2ForLatentConnectorNew were not initialized from the model checkpoint at gpt2-xl and are newly initialized: ['h.48.ln_2.weight', 'h.48.attn.c_proj.bias', 'h.48.mlp.c_proj.weight', 'h.48.mlp.c_fc.bias', 'h.48.attn.c_attn.bias', 'linear.weight', 'h.48.ln_1.bias', 'linear_emb.weight', 'h.48.attn.c_proj.weight', 'h.48.mlp.c_fc.weight', 'lm_head.bias', 'h.48.mlp.c_proj.bias', 'h.48.ln_1.weight', 'h.48.attn.c_attn.weight', 'h.48.ln_2.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForLatentConnectorAVG were not initialized from the model checkpoint at prajjwal1/bert-small and are newly initialized: ['bert.linear.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


VAE(
  (encoder): BertForLatentConnectorAVG(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, elementw

In [10]:
model_decoder()

GPT2ForLatentConnectorNew(
  (transformer): GPT2ModelForVAENew(
    (wte): Embedding(50260, 1600)
    (wpe): Embedding(1024, 1600)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-48): 49 x GPT2Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
    (linear): Linear(in_features=128, out_features=78400, bias=False)
    (linear_emb): Linear(in_features=128, out_features=1600, bias=False)
  )
  (lm_head): Lin

In [23]:
for i in train_dataloader:
    x0,x1,x_lengths = i
    break
max_len_values, _ = x_lengths.max(0)
x0 = x0[:, :max_len_values[0]]
x1 = x1[:, :max_len_values[1]]
x0 = x0.to("cuda")
x1 = x1.to("cuda")
x_lengths = x_lengths.to("cuda")
context_tokens = tokenizer_decoder.encode(tokenizer_decoder.bos_token)
attention_mask = (x0 != tokenizer_encoder.pad_token_id).float()
reconstruction_mask = (x1 != tokenizer_decoder.pad_token_id).float()
pooled_hidden_fea = model_vae.encoder(x0, attention_mask)[1]

mean, logvar = model_vae.encoder.linear(pooled_hidden_fea).chunk(2, -1)

latent_z = mean.squeeze(1)


In [30]:
out = sample_sequence_conditional(
                        model=model_vae.decoder,
                        context=context_tokens,
                        past=latent_z,
                        length=x_lengths[0, 1],  # Chunyuan: Fix length; or use <EOS> to complete a sentence
                        num_samples=latent_z.size(0),
                        device="cuda",
                        decoder_tokenizer=model_vae.tokenizer_decoder,
                        eos_id=model_vae.eos_token_id
                    )

In [31]:
out

tensor([[50258,   376,   376,   376,   376,   376,   376,   376,   317,   317,
           317,   317,   938,   938,    11,    11],
        [50258,   376,   376,   376,   376,   376,   376,   376,   376,   367,
           367,   367,   367,   367,   367,   367]], device='cuda:0')

In [35]:
tokenizer_decoder.decode(out[1])

'<BOS> F F F F F F F F H H H H H H H'

In [26]:
out = model_decoder(input_ids=x1, past=latent_z, labels=x1, label_ignore=tokenizer_decoder.pad_token_id)

CausalLMOutputWithCrossAttentions(loss=tensor([144.5167,  70.6147], device='cuda:0', grad_fn=<SumBackward1>), logits=tensor([[[ 3.6263,  2.4437, -0.8014,  ...,  0.3664, -3.3789, -0.4426],
         [ 3.6834,  2.3090, -1.8633,  ...,  0.4226, -3.2814,  0.1556],
         [ 3.6290,  2.0886, -2.5023,  ...,  0.4332, -3.2032,  0.4464],
         ...,
         [ 3.5221,  3.7896, -2.2201,  ..., -0.3465, -2.6947, -0.4070],
         [ 3.3350,  3.9541, -1.8440,  ..., -0.5015, -2.3233, -0.6540],
         [ 3.2172,  3.6478, -1.8716,  ..., -0.5604, -1.9235, -0.5699]],

        [[ 4.1487,  1.3632, -0.9358,  ..., -0.0980, -3.2885, -0.5068],
         [ 4.1680,  1.2141, -1.7426,  ..., -0.2307, -3.2480, -0.0675],
         [ 4.0657,  1.0321, -2.3216,  ..., -0.3363, -3.1668,  0.1866],
         ...,
         [ 3.7738,  1.6618, -3.9054,  ..., -0.5750, -2.3978,  1.0498],
         [ 3.8472,  1.7656, -3.8371,  ..., -0.6276, -2.2989,  1.0319],
         [ 3.9241,  1.8583, -3.7637,  ..., -0.6858, -2.2053,  1.0075]]],